<a href="https://colab.research.google.com/github/RubenCarty/Master-Artificial-Intelligence-UNIR/blob/main/DeepLearningClaseModelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Implementación de Reconocimiento Facial
## [M.Sc. Ruben Quispe](https://www.linkedin.com/in/ruben-quispe-l/)
# Paso 1: Configuración del Entorno

In [ ]:
# Instalar TensorFlow
!pip install tensorflow


# Paso 2: Importar Librerías Necesarias

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Paso 3: Cargar y Preprocesar Datos
Utilizaremos el conjunto de datos LFW, que contiene imágenes de rostros etiquetados.

In [ ]:
# Cargar el dataset de LFW
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

# Extraer imágenes y etiquetas
X = lfw_people.images
y = lfw_people.target

# Normalizar los datos
X = X / 255.0

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Redimensionar los datos para que se adapten a la entrada de la red neuronal
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)


# Paso 4: Construir el Modelo de Red Neuronal

In [ ]:
# Construir el modelo de red neuronal
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(lfw_people.target_names), activation='softmax')
])

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


# Paso 5: Entrenar el Modelo

In [ ]:
# Entrenar el modelo
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)


# Paso 6: Evaluar el Modelo

In [ ]:
# Evaluar el modelo en el conjunto de test
test_loss, test_acc = model.evaluate(X_test, y_test)
print('\nTest accuracy:', test_acc)


# Paso 7: Realizar Predicciones

In [ ]:
# Realizar predicciones
predictions = model.predict(X_test)

# Mostrar algunas predicciones
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array[i], true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'blue'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(lfw_people.target_names[predicted_label],
                                         100*np.max(predictions_array),
                                         lfw_people.target_names[true_label]),
                                         color=color)

# Graficar algunas imágenes con sus predicciones
num_rows = 5
num_cols = 3
num_images = num_rows * num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
    plt.subplot(num_rows, 2*num_cols, 2*i+1)
    plot_image(i, predictions, y_test, X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2]))
plt.tight_layout()
plt.show()
